### Imports

In [30]:
import numpy as np
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
import sys

# 1. Data Exploration

### Load metadata

In [31]:
df_meta = pd.read_csv('data/train_small-db.meta',delimiter='\t')

In [32]:
df_meta

,sequence.id,genome.id,taxid.strain,taxid.species,taxid.genus
0,Cupriavidus_necator_N-1,NC_015723,1042878,106590,106589
1,Cupriavidus_necator_N-1,NC_015724,1042878,106590,106589
2,Cupriavidus_necator_N-1,NC_015726,1042878,106590,106589
3,Cupriavidus_necator_N-1,NC_015727,1042878,106590,106589
4,Rhodospirillum_rubrum_ATCC_11170,NC_007641,269796,1085,1081
...,...,...,...,...,...
1559,Burkholderia_cenocepacia_J2315,NC_011002,216591,95486,32008
1560,Burkholderia_cenocepacia_J2315,NC_011003,216591,95486,32008
1561,Burkholderia_cenocepacia_MC0-3,NC_010508,406425,95486,32008
1562,Burkholderia_cenocepacia_MC0-3,NC_010512,406425,95486,32008


### Explore metadata

In [33]:
print('Number of sequence ids:',len(df_meta['sequence.id'].unique()))
print('Number of genome ids:',len(df_meta['genome.id'].unique()))
print('Number of taxid strains:',len(df_meta['taxid.strain'].unique()))
print('Number of taxid species:',len(df_meta['taxid.species'].unique()))
print('Number of taxid genus:',len(df_meta['taxid.genus'].unique()))

Number of sequence ids: 872
Number of genome ids: 1564
Number of taxid strains: 872
Number of taxid species: 193
Number of taxid genus: 76


### Load taxid data

In [34]:
df_taxid = pd.read_csv('data/train_small-db.species-level.taxid',header = None)

In [35]:
df_taxid

,0
0,106590
1,106590
2,106590
3,106590
4,1085
...,...
1559,95486
1560,95486
1561,95486
1562,95486


### Load sequence data

In [36]:
# %%time

# filename='/Users/ryanqnelson/Downloads/large-scale-metagenomics-1.0/data/train-dataset/train_small-db.fasta'

# count = 0
# for seq_record in SeqIO.parse(filename, "fasta"):
#     count += 1

# count

# 2. Build toy dataset

### Calculate sequence lengths
To identify small sequences which can be used to generate a toy dataset.

In [37]:
%%time

# create dataframe containing sequence lengths for each sequence
filename='/Users/ryanqnelson/Downloads/large-scale-metagenomics-1.0/data/train-dataset/train_small-db.fasta'

ids = []
lengths = []

for i,seq_record in enumerate(SeqIO.parse(filename, "fasta")):
        ids.append(seq_record.id)
        lengths.append(len(seq_record.seq))
        
df_lengths = pd.DataFrame(list(zip(ids, lengths)), columns =['genome.id', 'sequence.length'])
df_lengths.head()

CPU times: user 15 s, sys: 1.4 s, total: 16.4 s
Wall time: 16.5 s


,genome.id,sequence.length
0,NC_015723,2684606
1,NC_015724,424140
2,NC_015726,3872936
3,NC_015727,1499175
4,NC_007641,53732


In [38]:
# join with taxonomy information to get species
df = df_meta.set_index('genome.id').join(
    df_lengths.set_index('genome.id'),lsuffix='_caller', rsuffix='_other'
).reset_index()

In [39]:
df.head()

,genome.id,sequence.id,taxid.strain,taxid.species,taxid.genus,sequence.length
0,NC_015723,Cupriavidus_necator_N-1,1042878,106590,106589,2684606
1,NC_015724,Cupriavidus_necator_N-1,1042878,106590,106589,424140
2,NC_015726,Cupriavidus_necator_N-1,1042878,106590,106589,3872936
3,NC_015727,Cupriavidus_necator_N-1,1042878,106590,106589,1499175
4,NC_007641,Rhodospirillum_rubrum_ATCC_11170,269796,1085,1081,53732


In [40]:
# df.to_csv('data/train_small-db.lengths.csv')

In [41]:
# bin data v1
bins = [0, 1e3, 1e4, 1e5, 1e6, 1e7, 1e8, 1e9, 1e10]
df['binned'] = pd.cut(df['sequence.length'], bins)

In [42]:
df['binned'].value_counts()

(1000000.0, 10000000.0]          953
(10000.0, 100000.0]              291
(1000.0, 10000.0]                198
(100000.0, 1000000.0]            122
(1000000000.0, 10000000000.0]      0
(100000000.0, 1000000000.0]        0
(10000000.0, 100000000.0]          0
(0.0, 1000.0]                      0
Name: binned, dtype: int64

In [43]:
# bin data v2
bins = [0, 1e3, 2*1e3, 5*1e3, 1e4, 1e5, 1e6, 1e7]
df['binned'] = pd.cut(df['sequence.length'], bins)

In [44]:
df['binned'].value_counts()

(1000000.0, 10000000.0]    953
(10000.0, 100000.0]        291
(5000.0, 10000.0]          134
(100000.0, 1000000.0]      122
(2000.0, 5000.0]            59
(1000.0, 2000.0]             5
(0.0, 1000.0]                0
Name: binned, dtype: int64

In [45]:
df[df['sequence.length'] < 2000]

,genome.id,sequence.id,taxid.strain,taxid.species,taxid.genus,sequence.length,binned
36,NC_013451,Staphylococcus_aureus_subsp._aureus_ED98,681288,1280,1279,1442,"(1000.0, 2000.0]"
540,NC_006375,Lactobacillus_plantarum_WCFS1,220668,1590,1578,1917,"(1000.0, 2000.0]"
757,NC_019565,Helicobacter_pylori_Aklavik86,1055532,210,209,1634,"(1000.0, 2000.0]"
818,NC_015407,Mycoplasma_mycoides_subsp._capri_LC_str._95010,862259,2102,2093,1840,"(1000.0, 2000.0]"
1197,NC_016841,Klebsiella_pneumoniae_subsp._pneumoniae_HS11286,1125630,573,570,1308,"(1000.0, 2000.0]"


In [46]:
# get sequences with lengths between 1000 and 2000 bp
genome_ids = df[df['sequence.length'] < 2000]['genome.id'].to_list()
species_ids = df[df['sequence.length'] < 2000]['taxid.species'].to_list()

In [47]:
print(genome_ids)
print(species_ids)

['NC_013451', 'NC_006375', 'NC_019565', 'NC_015407', 'NC_016841']
[1280, 1590, 210, 2102, 573]


### Select specific samples
Select smallest length samples to use for method development.

In [48]:
# input_file='/Users/ryanqnelson/Downloads/large-scale-metagenomics-1.0/data/train-dataset/train_small-db.fasta'
# output_file = "data/train_small-db_toy-2000.fasta"

# # get toy sample sequences
# sequences = []
# for seq_record in SeqIO.parse(input_file, "fasta"):
#     if seq_record.id in genome_ids:
#         print(seq_record.id, len(seq_record.seq))
#         sequences.append(seq_record)
        
# # write toy sample to file
# with open(output_file, "w") as output_handle:
#     SeqIO.write(sequences, output_handle, "fasta")
    

In [49]:
# # verify this worked by reading results back in
# input_file = "data/train_small-db_toy-2000.fasta"
# for seq_record in SeqIO.parse(input_file, "fasta"):
#     print(seq_record.id, len(seq_record.seq))

### Save matching taxid.species

In [50]:
# genome_ids = df[df['sequence.length'] < 2000]['taxid.species'].to_list()
# print(genome_ids)
# output_file = "data/train_small-db_toy-2000.taxid"
# with open(output_file,'w') as output_handle:
#     for each in genome_ids:
#         output_handle.write(str(each) + '\n')

# 2. Build small scale dataset
around 100 samples

In [51]:
len(df[df['sequence.length'] < 10000])

198

In [52]:
# get sequences with lengths between 1000 and 2000 bp
genome_ids = df[df['sequence.length'] < 10000]['genome.id'].to_list()
species_ids = df[df['sequence.length'] < 10000]['taxid.species'].to_list()

In [19]:
# input_file='/Users/ryanqnelson/Downloads/large-scale-metagenomics-1.0/data/train-dataset/train_small-db.fasta'
# output_file = "data/train_small-db_toy-10000.fasta"

# # get toy sample sequences
# sequences = []
# for seq_record in SeqIO.parse(input_file, "fasta"):
#     if seq_record.id in genome_ids:
# #         print(seq_record.id, len(seq_record.seq))
#         sequences.append(seq_record)
        
# # write toy sample to file
# with open(output_file, "w") as output_handle:
#     SeqIO.write(sequences, output_handle, "fasta")
    

In [55]:
# # verify this worked by reading results back in
# input_file = "data/train_small-db_toy-10000.fasta"
# for i,seq_record in enumerate(SeqIO.parse(input_file, "fasta")):
# #     print(seq_record.id, len(seq_record.seq))
#     assert seq_record.id == genome_ids[i], seq_record.id + ' '  + str(i)

NC_021657 2908
NC_022227 3332
NC_022228 2472
NC_006629 4440
NC_017346 2241
NC_013451 1442
NC_013452 4440
NC_017348 2993
NC_017334 5246
NC_017335 4381
NC_017336 3158
NC_017332 3011
NC_007790 3125
NC_007791 4439
NC_012417 3125
NC_022605 3011
NC_008500 4449
NC_008501 3361
NC_021989 9319
NC_021990 2189
NC_021996 4072
NC_000957 5228
NC_001904 9386
NC_017399 8722
NC_020273 8009
NC_017189 6758
NC_022531 8438
NC_011657 3091
NC_011776 3091
NC_007104 5108
NC_007106 8191
NC_007107 9150
NC_018494 3091
NC_016773 5436
NC_016774 3091
NC_016793 3869
NC_010008 5428
NC_010009 9098
NC_017194 5838
NC_018881 8240
NC_018882 7635
NC_018883 6880
NC_018884 5518
NC_018885 2062
NC_018886 8513
NC_018490 8574
NC_018503 9070
NC_018511 7697
NC_018512 6824
NC_017207 8513
NC_017210 6880
NC_017211 8252
NC_017212 9547
NC_020241 8513
NC_020242 8241
NC_020243 7635
NC_020377 8251
NC_020378 9671
NC_020390 6880
NC_012657 9953
NC_021515 7240
NC_021520 6464
NC_021525 8636
NC_021228 8686
NC_006375 1917
NC_006376 2365
NC_021498 

In [58]:
# species_ids = df[df['sequence.length'] < 10000]['taxid.species'].to_list()

# output_file = "data/train_small-db_toy-10000.taxid"
# with open(output_file,'w') as output_handle:
#     for each in species_ids:
#         output_handle.write(str(each) + '\n')

In [60]:
# df[df['genome.id'] == 'NC_017288']

,genome.id,sequence.id,taxid.strain,taxid.species,taxid.genus,sequence.length,binned
1513,NC_017288,Chlamydia_psittaci_6BC_(Chlamydophila_psittaci...,331636,83554,810,7553,"(5000.0, 10000.0]"
